In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_serving.obt
USING DELTA
PARTITIONED BY (run_time)
AS
SELECT
  cr.*,
  dm.sop,
  dm.method_id,
  dm.meth_id,
  dm.temp_id,
  dm.report_template_version,
  dm.seq_id,
  dm.seq_version,
  ds.equipment_id,
  ds.im_id,
  ds.column_id,
  du.submit_id,
  du.submit_datetime,
  du.review_id,
  du.review_datetime,
  du.approval_id,
  du.approval_datetime,
  du.status
FROM workspace.gold_serving.run_sst_result cr
LEFT JOIN workspace.gold_serving.dimension_methods dm
  ON cr.pk = dm.method_id OR cr.run_time = dm.time
LEFT JOIN workspace.gold_serving.dimension_systems ds
  ON cr.pk = ds.pk OR cr.run_time = ds.time
LEFT JOIN workspace.gold_serving.dimension_users du
  ON cr.pk = du.pk OR cr.run_time = du.time;


MERGE INTO workspace.gold_serving.obt AS target
USING (
  SELECT * FROM (
    SELECT
      cr.*,
      dm.sop,
      dm.method_id,
      dm.meth_id,
      dm.temp_id,
      dm.report_template_version,
      dm.seq_id,
      dm.seq_version,
      ds.equipment_id,
      ds.im_id,
      ds.column_id,
      du.submit_id,
      du.submit_datetime,
      du.review_id,
      du.review_datetime,
      du.approval_id,
      du.approval_datetime,
      du.status,
      ROW_NUMBER() OVER (
        PARTITION BY cr.pk, cr.samp_id, cr.sst_number
        ORDER BY cr.run_time DESC
      ) AS rn
    FROM workspace.gold_serving.run_sst_result cr
    LEFT JOIN workspace.gold_serving.dimension_methods dm
      ON cr.pk = dm.method_id OR cr.run_time = dm.time
    LEFT JOIN workspace.gold_serving.dimension_systems ds
      ON cr.pk = ds.pk OR cr.run_time = ds.time
    LEFT JOIN workspace.gold_serving.dimension_users du
      ON cr.pk = du.pk OR cr.run_time = du.time
  ) WHERE rn = 1
) AS source
ON target.pk = source.pk
AND target.run_time = source.run_time
AND target.samp_id = source.samp_id
AND target.sst_number = source.sst_number
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *;

